In [1]:
import GPflow
import tensorflow as tf

ImportError: No module named GPflow

In [2]:
class assignGP(GPflow.model.GPModel):
    def __init__(self, X, Y, kern, indices):
        GPflow.model.GPModel.__init__(self, X, Y, kern, likelihood=GPflow.likelihoods.Gaussian(), mean_function=GPflow.mean_functions.Zero())
        self.Phi = ProbMatrix(indices)
        self.A = np.random.randn(*self.Phi.shape)
        
    def build_likelihood(self):
        N = self.Y.shape[0]
        M = self.X.shape[0]
        
        K = self.kern.K(self.X)
        L = tf.cholesky(K)
        tmp = tf.user_ops.triangular_solve(L, GPflow.tf_hacks.eye(M), 'lower')
        Ki = tf.user_ops.triangular_solve(tf.transpose(L), tmp, 'upper')
        tau = 1./self.likelihood.variance
        
        
        Phi = self.Phi.make_matrix()
        A = tf.matmul(tf.transpose(Phi), Phi)
        A = A - tf.diag(tf.user_ops.get_diag(A))
        A = A + tf.diag(tf.reduce_sum(Phi, 0))
        
        Lamb = A * tau + Ki
        R = tf.cholesky(Lamb)
        PhiY = tf.matmul(Phi, self.Y)
        return -0.5*N*tf.log(tau)\
               - 0.5*tf.reduce_sum(tf.log(tf.user_ops.get_diag(L)))\
               - 0.5*tf.reduce_sum(tf.log(tf.user_ops.get_diag(R)))\
               - 0.5*tau*tf.reduce_sum(tf.square(self.Y))\
               - 0.5*tf.reduce_sum(tf.square(tau * tf.user_ops.triangular_solve(R, PhiY, 'lower')))
    

In [22]:
def flat_softmax(x):
    return tf.reshape(tf.nn.softmax(tf.reshape(x, (1,-1))), (-1))

class ProbMatrix(GPflow.param.Parameterized):
    def __init__(self, inds):
        GPflow.param.Parameterized.__init__(self)
        self.shape = ( len(inds), np.max(np.hstack(inds)) +1 )
        
        self.log_phis = GPflow.param.Param(np.random.randn(np.hstack(inds).size*10))
        self.inds = inds
    
    def make_matrix(self):
        return self.make_matrix_dumb()
    
    def make_matrix_sparse(self):
        """
        This works, but the sparse matrix object in tensorflow 
        is lacking features, like multiplication with dense 
        matrices, and gradients!
        
        use make_matrix_dumb instead
        """
        #softmax each row of the matrix and concatenate
        count = 0
        probs = []
        for r, ind in enumerate(inds):
            n = len(ind)
            probs.append(flat_softmax(self.log_phis[count:count+n]))
        probs = tf.concat(0, probs)
        
        #build a tf style index
        i = np.hstack(inds)
        i = np.vstack((np.arange(i.size), i)).T
        
        #make a sparse matrix
        Z = tf.SparseTensor(i, probs, self.shape)
        
        return Z
    
    def make_matrix_dumb(self):
        #softmax each row of the matrix, and put in zeros
        rows = []
        count = 0
        for ind in self.inds:
            n = len(ind)
            probs = flat_softmax(self.log_phis[count:count+n])
            count += n
            
            #make the row one piece at a time (!)
            row = []
            rowcount = 0
            for i in range(n):
                row.append( tf.zeros((ind[i] - rowcount,), tf.float64))
                rowcount = ind[i]+1
                row.append(probs[i:i+1])
            row.append(tf.zeros((self.shape[1] - rowcount,), tf.float64))
            row = tf.concat(0, row)
            row = tf.expand_dims(row, 0)
            rows.append(row)
            
        return rows[0]#tf.concat(0, rows)               


In [23]:
inds = [[0,1],[2,3]]

In [24]:
X = np.random.randn(4,1)
Y = np.random.randn(4,1)
k = GPflow.kernels.Matern32(1) + GPflow.kernels.White(1)
m = assignGP(X, Y, k, inds)

In [25]:

m._compile()


compiling tensorflow function...
done


In [26]:
m._objective(m.get_free_state())

InvalidArgumentError: sizes input must be 1-D, not shape []
	 [[Node: Reshape_43 = Reshape[T=DT_DOUBLE, _device="/job:localhost/replica:0/task:0/cpu:0"](Softmax_8, Reshape_43/shape)]]
Caused by op u'Reshape_43', defined at:
  File "/Users/james/anaconda/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/james/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/james/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/james/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 592, in launch_instance
    app.start()
  File "/Users/james/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 403, in start
    ioloop.IOLoop.instance().start()
  File "/Users/james/anaconda/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).sta

In [27]:
m.make_tf_array(m._free_vars)
with m.tf_mode():
    P = m.Phi.make_matrix()

In [30]:
m._session.run(P, feed_dict={m._free_vars:m.get_free_state()})

InvalidArgumentError: sizes input must be 1-D, not shape []
	 [[Node: Reshape_52 = Reshape[T=DT_DOUBLE, _device="/job:localhost/replica:0/task:0/cpu:0"](Softmax_10, Reshape_52/shape)]]
Caused by op u'Reshape_52', defined at:
  File "/Users/james/anaconda/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/james/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/james/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/james/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 592, in launch_instance
    app.start()
  File "/Users/james/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 403, in start
    ioloop.IOLoop.instance().start()
  File "/Users/james/anaconda/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).st